# **GROUP 1: MOSAICS**

### **Done by Pere Maeso, Sergi Escudero, David Miquel, Oriol Ramos and Pepe Moran**




**Write the name of the members of the group:**

#

Our code creates a **photomosaic**, an image composed of smaller images (tiles) that resemble the original picture when viewed from a distance. It analyzes an image, divides it into small sections, and replaces each section with the most color-matching image from a predefined dataset.

## **Key Parts of the Code**

**Importation:**
We proceed to import the necessary libraries in order to make the code work.

In [19]:
import json  # To handle caching of data in JSON format
import os  # To interact with the operating system (listing files, checking file existence, etc.)
import math  # For mathematical operations, such as calculating distances
import random  # To randomly select elements

import numpy as np  # To handle operations with matrices and numerical arrays
import cv2  # For image processing
from PIL import Image
import matplotlib.pyplot as plt
import copy
from scipy.ndimage import zoom

**Average Color Calculation:**
*The function get_average_color(img)* calculates the average color of an image by averaging pixel values. This helps in matching tiles to the closest images in the dataset.

In [20]:
# Function to calculate the average color of an image
def get_average_color(img):
    
    return img[0, 0]
    

**Finding the Closest Color:**
The function *get_closest_color(color, colors)* uses Euclidean distance to compare colors and find the closest match from a predefined list. This ensures the tiles resemble the original image.

In [26]:
# Function to find the closest color from a predefined list of colors
def get_closest_color(color, colors):
    if not colors:  # Si la lista está vacía, lanzar un error o devolver un color predeterminado
        raise ValueError("No hay colores en la caché")

    color = np.array(color)
    colors = np.array([np.array(eval(c)) for c in colors])  # Convertir los colores a arrays NumPy

    distances = np.linalg.norm(colors - color, axis=1)  # Calcular distancia euclidiana
    return str(tuple(colors[np.argmin(distances)]))


**Caching Image Data:**
If a cache does not exist, the program scans the "animals" directory, computes the average color for each image, and stores the results in a cache.json file. This avoids recalculating colors every time the program runs.


In [27]:
def cache(carpeta_script, carpeta_fotos):
    print("Generating cache...")
    images = []

    for root, dirs, files in os.walk(carpeta_fotos):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                file = os.path.join(root, file)
                file = os.path.relpath(file, carpeta_script)
                images.append(file)

    data = {}  # Dictionary to store average colors and their associated images

    for img_path in images:
        imagen = Image.open(img_path)  # Cargar imagen JPEG
        imagen_np = np.array(imagen)
        average_color = get_average_color(imagen_np)  # Get the average color of the image

        # Store image paths based on their average color in the dictionary
        if str(tuple(average_color)) in data:
            data[str(tuple(average_color))].append(str(img_path))
        else:
            data[str(tuple(average_color))] = [str(img_path)]

    # Save the dictionary to a JSON file to avoid recalculating in the future
    with open("cache.json", "w") as file:
        json.dump(data, file, indent=2, sort_keys=True)

    print("Cache generated")

## **Example with our Image**


Generate the cache

In [5]:
carpeta_fotos = r"C:\Users\perem\Presentacio PSIV\animals"
carpeta_script = r"C:\Users\perem\Presentacio PSIV"
cache(carpeta_script, carpeta_fotos)

Generating cache...
Cache generated


Load the cached data with average colors and image paths

In [6]:
with open("cache.json", "r") as file:
    data = json.load(file)
    
print("Data loaded correctly")

Data loaded correctly


Example with our image:

In [ ]:

path_imatge =  # Introduce the image path
    
imatge_original = cv2.imread(path_imatge)
imatge_original = cv2.cvtColor(imatge_original, cv2.COLOR_BGR2RGB)
img = copy.deepcopy(imatge_original)

tile_height =  # Introduce the tile height
tile_width =  # Introduce the tile width

scale_factor =  # Introduce the scale factor for better results

if scale_factor != 1:
    img_height, img_width = img.shape[:2]
    new_width = int(img_width * scale_factor)  
    new_height = int(img_height * scale_factor) 
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

img_height, img_width, _ = img.shape
num_tiles_h, num_tiles_w = img_height // tile_height, img_width // tile_width
img = img[:tile_height * num_tiles_h, :tile_width * num_tiles_w]

tiles = []
for y in range(0, img_height, tile_height):
    for x in range(0, img_width, tile_width):
        tiles.append((y, y + tile_height, x, x + tile_width))

for tile in tiles:
    y0, y1, x0, x1 = tile
    try:
        average_color = get_average_color(img[y0:y1, x0:x1])
    except Exception:
        continue
    closest_color = get_closest_color(average_color, data.keys())
    i_path = random.choice(data[str(closest_color)])
    imagen = Image.open(i_path)
    i = np.array(imagen)
    i = cv2.resize(i, (tile_width, tile_height))

    img[y0:y1, x0:x1] = i

    cv2.imshow('Mosaic', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    cv2.waitKey(1)

cv2.waitKey(0)
cv2.destroyAllWindows() 

plt.imsave("output.png", img)

fig, axes = plt.subplots(1, 2, figsize=(12,8))

axes[0].imshow(imatge_original, cmap="gray")
axes[0].set_title("Foto original:")
axes[0].axis('off')
axes[1].imshow(img, cmap="gray")
axes[1].set_title("Mosaic:")
axes[1].axis('off')

plt.show()

# **CHALLENGE**

**STEP 1: Process the Base Image:**
Load YOUR IMAGE and divide it into small tiles (we recommend(10x10 pixels)). This will ensure the image size is adjusted to fit these tiles exactly.

**STEP 2: Replace Tiles with Matching Images:**
Iterate through each tile, calculate its average color, and find the closest match from the cached images. A random image from the matching color group should be resized and placed into the mosaic.

**STEP 3: Fix the *get_average_color()* function:**
Fix the function making it to compute the average of the image and not the first pixel of it.

**STEP 4: Fix the *get_closest_color()* function:**
Fix the function in order to calculate the closest color based on the euqlidean distance instead of choosing a random image.

**FINAL STEP: Real-Time Display and Final Output:**
As each tile is replaced, the program updates the display *(cv2.imshow)* in real-time. Once all tiles are replaced, the final photomosaic is saved as *output.jpg*.


In [8]:
# WRITE YOUR CODE HERE

In [28]:
carpeta_fotos = r"C:\Users\LAURA\OneDrive\Escriptori\PSIV\animals1"
carpeta_script = r"C:\Users\LAURA\OneDrive\Escriptori\PSIV"
cache(carpeta_script, carpeta_fotos)

Generating cache...
Cache generated


In [29]:
with open("cache.json", "r") as file:
    data = json.load(file)
    
print("Data loaded correctly")

Data loaded correctly


In [30]:
path_imatge =  "funny_cat.jpg" # Introduce the image path
    
imatge_original = cv2.imread(path_imatge)
imatge_original = cv2.cvtColor(imatge_original, cv2.COLOR_BGR2RGB)
img = copy.deepcopy(imatge_original)

tile_height =  10 # Introduce the tile height
tile_width =  10 # Introduce the tile width

scale_factor =  1 # Introduce the scale factor for better results

if scale_factor != 1:
    img_height, img_width = img.shape[:2]
    new_width = int(img_width * scale_factor)  
    new_height = int(img_height * scale_factor) 
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

img_height, img_width, _ = img.shape
num_tiles_h, num_tiles_w = img_height // tile_height, img_width // tile_width
img = img[:tile_height * num_tiles_h, :tile_width * num_tiles_w]

tiles = []
for y in range(0, img_height, tile_height):
    for x in range(0, img_width, tile_width):
        tiles.append((y, y + tile_height, x, x + tile_width))

for tile in tiles:
    y0, y1, x0, x1 = tile
    try:
        average_color = get_average_color(img[y0:y1, x0:x1])
    except Exception:
        continue
    closest_color = get_closest_color(average_color, data.keys())
    i_path = random.choice(data[str(closest_color)])
    imagen = Image.open(i_path)
    i = np.array(imagen)
    i = cv2.resize(i, (tile_width, tile_height))

    img[y0:y1, x0:x1] = i

    cv2.imshow('Mosaic', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    cv2.waitKey(1)

cv2.waitKey(0)
cv2.destroyAllWindows() 

plt.imsave("output.png", img)

fig, axes = plt.subplots(1, 2, figsize=(12,8))

axes[0].imshow(imatge_original, cmap="gray")
axes[0].set_title("Foto original:")
axes[0].axis('off')
axes[1].imshow(img, cmap="gray")
axes[1].set_title("Mosaic:")
axes[1].axis('off')

plt.show()

ValueError: No hay colores en la caché